In [1]:
from __future__ import division
import numpy as np
from sklearn.datasets import make_classification
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext line_profiler

In [7]:
from sklearn.datasets import load_iris
data = load_iris()

In [8]:
X = data.data
X.shape

(150, 4)

In [9]:
y = data.target
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

### STRUT fcns

In [43]:
import math
from math import log

def convert_from_scikit_learn_to_dic_ite(node_index,is_leaves, children_left,children_right,feature,threshold,value,labels,C):
        
        a = is_leaves[0]
        b = feature[0]
        c = threshold[0]
        if (a):
            d = value[0]  #datapoints of each class in the node
            d2 = np.squeeze(d/np.sum(d))
            d3 = np.zeros(C)
            d3[labels] = d2
            e = labels[np.argmax(d2)]
            return {'splitting_feature' : None,
            'left' : None,
            'right' : None,
            'is_leaf': True,
            'prediction': e,
            'labels_distribution':d3}
    
        else:
            left = children_left[0]-node_index[0]
            if(left==-1):
                left_tree = None
            else:
                left = int(left)
                left_tree = convert_from_scikit_learn_to_dic_ite(node_index[left:],is_leaves[left:], children_left[left:],children_right[left:],feature[left:],threshold[left:],value[left:],labels,C)
            right = children_right[0]-node_index[0]
            if(right==-1):
                right_tree = None
            else:
                right = int(right)
                right_tree = convert_from_scikit_learn_to_dic_ite(node_index[right:],is_leaves[right:], children_left[right:],children_right[right:],feature[right:],threshold[right:],value[right:],labels,C)
            return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': b,
            'threshold'        : c,
            'left'             : left_tree, 
            'right'            : right_tree,
            'labels_distribution': None}


def convert_from_scikit_learn_to_dic_STRUT(tree,threshold,C,Q,target_lf,target_lr,feature):
    # C is the size of the whole labels
    # labels are the labels that are used in the this tree
    labels = range(0,C,1)
    #n_nodes = tree.tree_.node_count
    n_nodes = target_lr.shape[0]
    children_left = target_lf
    children_right = target_lr
    #feature = tree.tree_.feature
    node_index = np.array(range(0,n_nodes))
    Val = Q   #datapoints in node
# The tree structure can be traversed to compute various properties such
# as the depth of each node and whether or not it is a leaf.
    node_depth = np.zeros(shape=n_nodes)
    is_leaves = np.zeros(shape=n_nodes, dtype=bool)
    stack = [(0, -1)]  # seed is the root node id and its parent depth
    while len(stack) > 0:
        node_id, parent_depth = stack.pop()
        node_depth[node_id] = parent_depth + 1
    # If we have a test node
        if (children_left[node_id] != children_right[node_id]):
            stack.append((children_left[node_id], parent_depth + 1))
            stack.append((children_right[node_id], parent_depth + 1))
        else:
            is_leaves[node_id] = True
    
    return convert_from_scikit_learn_to_dic_ite(node_index,is_leaves, children_left,children_right,feature,threshold,Val,labels,C)

def TransferToSKLsequence(ch_left,ch_right,subset):
    new_left = np.zeros(len(subset))
    new_right = np.zeros(len(subset))
    for i in range(len(subset)):
        (I,) = np.where(subset == ch_left[subset[i]])
        if len(I)==0:
            new_left[i] = -1
        else:
            
            new_left[i] = I
            
        (I,) = np.where(subset == ch_right[subset[i]])
        if len(I)==0:
            new_right[i] = -1
        else:
            new_right[i] = I
    return (new_left,new_right)

def kl (p,q): # Kullback-libler divegence
    p = np.asarray(p, dtype=np.float)
    q = np.asarray(q, dtype=np.float)+.0001
    return np.sum(np.where(p != 0,p * np.log10((p / q)), 0))

def jsd(p,q): # Symmetric Kullback-libler divergence
    p = np.asarray(p, dtype=np.float)
    q = np.asarray(q, dtype=np.float)
    m = (p+q)/2
    return (kl(p,m)+kl(m,q))/2

def infogain(yleft,len_left,yright,len_right):
    yparent = (yleft+yright)/2
    N = len_left+len_right
    #compute information gain
    I = entropy(yparent) -( (len_left/N)*entropy(yleft) + (len_right/N)*entropy(yright) )   
    return I

#entropy for multiple classes
def entropy(y):
    y1 = y[y!=0]
    H = -(y1*np.log10(y1)).sum()
    return H 

def partition(Xtarget,ytarget,index_of_data,feature,C,threshold): # divide the data to the left and rightbased on the threshold
    left = index_of_data[Xtarget[index_of_data,feature]<threshold]
    if(len(left)==0):
        left = index_of_data[Xtarget[index_of_data,feature]<=threshold]
    labels_left = ytarget[left]
    qL = np.bincount(labels_left)
    right = index_of_data[Xtarget[index_of_data,feature]>=threshold]
    labels_right = ytarget[right]
    qR = np.bincount(labels_right)
    qR = np.append(qR,np.zeros(np.max([C-qR.shape[0],0])))
    qL = np.append(qL,np.zeros(np.max([C-qL.shape[0],0]))) 
    qL = qL/qL.sum()
    qR = qR/qR.sum()
    return [qL,left,qR,right]

def dg(Sleft,lenleft,Sright,lenright,QL,QR): # DG function as in the paper    
    return 1-(lenleft/(lenleft+lenright))*jsd(Sleft,QL)-(lenright/(lenleft+lenright))*jsd(Sright,QR)
    
def threshold_selection(X,y,S,f,QL,QR,C,verbos): # finding the best threshold
    fvals = np.sort(X[S,f])
    num_data_points = len(fvals)
    N = 10
    Val  = np.array([])
    #Val_swap  = np.array([])
    Val_infogain = np.array([])
    if num_data_points > N-1: 
        I = range(0,num_data_points,np.floor(num_data_points/N).astype(int))
        fvals = fvals[I[1:-1]]
    for i in fvals: #looping through the thresholds
        [Sleft, left, Sright, right] = partition(X,y,S,f,C,i) #find splits based on threshold
        
        #fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, sharex='col', sharey='row')
        #ax1.plot(QL)
        #ax1.set_title('QL')
        #ax2.plot(Sleft, color='r')
        #ax2.set_title('QprimeL')
        #ax3.plot(QR)
        #ax3.set_title('QR')
        #ax4.plot(Sright, color='r')
        #ax4.set_title('QprimeR')
        Val = np.append(Val,dg(Sleft,len(left),Sright,len(right),QL,QR)) # Here we compute DG between the source dist and target
        #print 'DG = %s' %Val[-1]
        #print 'QL=%s, QR=%s, Sleft=%s, Sright=%s' %(QL,QR,Sleft,Sright)
        Val_infogain = np.append(Val_infogain,infogain(Sleft,len(left),Sright,len(right))) #Compute IG
        #Val_swap = np.append(Val_swap,dg(Sleft,len(left),Sright,len(right),QR,QL)) # this is the divergence measure for each threshold split  
    if(verbos):
        fig, (ax1, ax2, ax3) = plt.subplots(1, 3, sharex='col', sharey='row')
    #ax1.plot(Val)
    #ax1.set_title('DG')
    #ax2.plot(Val_infogain)
    #ax2.set_title('infogain')
        ax1.plot(fvals,Val,'r')
        ax1.hold(True)
        ax1.plot(fvals,Val_infogain)
        ax1.hold(False)
        ax1.set_title('DG and Infogain')
    #plt.show()
    Val[np.isnan(Val)] = min(Val[~np.isnan(Val)])
    Val_infogain[np.isnan(Val_infogain)] = min(Val_infogain[~np.isnan(Val_infogain)])
    #Val_swap[np.isnan(Val_swap)] = min(Val_swap[~np.isnan(Val_swap)])
    th = fvals[np.argmax(Val)] #Maximizing threshold for DG
    th_infogain = fvals[np.argmax(Val_infogain)]
    if(len(S)<50):
        [ql, left, qr, right] = partition(X,y,S,f,C,th_infogain)
    else:
        [ql, left, qr, right] = partition(X,y,S,f,C,th)
    if(verbos):
        ax2.plot(ql)
        ax2.hold(True)
        ax2.plot(qr)
        ax2.hold(False)
        ax2.set_title('Dist Target Data')

        ax3.plot(QL)
        ax3.hold(True)
        ax3.plot(QR)
        ax3.hold(False)
        ax3.set_title('Dist Source Data')
    #print Val
    return [th, ql, qr, left, right]

def classify(tree, x):   
    # if the node is a leaf node.
    if tree['is_leaf']:
        return tree['labels_distribution'] 
    else:
        # split on feature.
        val_split_feature = x[tree['splitting_feature']]
        if val_split_feature < tree['threshold']:
            return classify(tree['left'], x)
        else:
            return classify(tree['right'],x)

def forest_posterior(RF,x):

    T = len(RF)  #the number of trees 

    #infer the number of classes
    P0 = classify(RF[0],x)
    C = len(P0)
    
    Pt = np.zeros((T,C)) #matrix of posteriors from each tree (T x Nclasses)
    Pt[0,:] = P0
    for t in range(len(RF))[1:]:
        Pt[t,:] = classify(RF[t],x) 
    return Pt

#classify input based on majority voting of each tree prediction
def forest_classify_majority(RF,x):
        Pt = forest_posterior(RF,x)
        Yt = np.argmax(Pt,axis=1)         
        C,unique_counts = np.unique(Yt,return_counts=True) #the id of classes and number of each
        return C[np.argmax(unique_counts)]   
    
#classify input by averaging posteriors 
def forest_classify_ensemble(RF,x):
    Pt = forest_posterior(RF,x)
    Pforest = Pt.mean(axis=0)
    ypred = np.argmax(Pt.mean(axis=0))
    return ypred

def evaluate_classification_error(RF, X, y, method = None):  
    # Apply the forest_classify(RF, x) to each row in your data
    if method == None:
        ypred = map(lambda x: forest_classify_ensemble(RF,x), X)
        # Once you've made the predictions, calculate the classification error and return it
        mistakes = sum(ypred != y)
        error = mistakes/len(y)
    return error

def value_for_all(estimator,N):
    from scipy.sparse import csr_matrix
    ch_left = estimator.tree_.children_left
    ch_right = estimator.tree_.children_right
    (cl,) = np.where(ch_left!=-1)
    (cr,) = np.where(ch_right!=-1)
    cap = estimator.tree_.capacity
    dis_node = np.zeros((cap,estimator.tree_.n_classes))
    A = np.zeros([cap,cap])
    D = A
    A = csr_matrix(A)
    A[cl,ch_left[cl]] = 1
    A[cr,ch_right[cr]] = 1
    B = A
    C = B
    while(C.sum()!=0):
        C = A*C
        B = B + C
    I,J = B.nonzero()
    D[I,J] = 1
    (I,) = np.where(ch_left==-1)
    dis_node[I,:] = np.squeeze(estimator.tree_.value[I])
    for i in I:
        dis_node[i,:] = dis_node[i,:]/dis_node[i,:].sum()
    (remain1,) = np.where(ch_left!=-1)
    for i in remain1:
        (I,) = np.where(D[i,:]==1)
        dis_node[i,:] = np.sum(np.squeeze(estimator.tree_.value[I]),axis = 0)
        dis_node[i,:] = dis_node[i,:]/dis_node[i,:].sum()
    Dis_node = np.zeros((cap,N))
    Dis_node[:,estimator.classes_.astype(int)] = dis_node
    return Dis_node
    
def STRUCT(Xsource,ysource,Xtarget,ytarget,n_trees,C,verbos = False):
    # Assumption: ysource has all the labels of the problem 
    #estimator = DecisionTreeClassifier(max_features='sqrt',random_state=0,max_depth=2)
    Estimator = RandomForestClassifier(max_features='sqrt',random_state=0,n_estimators=n_trees)
    Estimator = Estimator.fit(Xsource, ysource)
    RF = []
    for rf in range(Estimator.n_estimators):   #can we change it to a parfor loop or map?
        estimator = Estimator.estimators_[rf]
        dis_node = value_for_all(estimator,C)
        P = list(np.zeros(estimator.tree_.capacity))
        P[0] = range(len(ytarget))
        Q = list(np.zeros(estimator.tree_.capacity))
        Q[0] = dis_node[0,:]
        thresh = np.zeros(estimator.tree_.capacity)
        remain = [0]
        subset = []
        while(len(remain)!=0):
            i = remain[0]
            LF = estimator.tree_.children_left
            LR = estimator.tree_.children_right
            index_left = LF[i]
            index_right = LR[i]
            if(index_left!=-1):
                QL = dis_node[index_left,:]
                QR = dis_node[index_right,:]
                f = estimator.tree_.feature[i]
                [th, ql, qr, left, right] = threshold_selection(Xtarget,ytarget,np.array(P[i]),f,QL,QR,C,verbos) 
                thresh[i] = th
                P[index_left] = left
                P[index_right] = right
                Q[index_left] = ql
                Q[index_right] = qr
                if(len(left)!=0):
                    remain = np.append(remain,index_left)
                if(len(right)!=0):
                    remain = np.append(remain,index_right)
                if(len(left)>0 and len(right)>0):
                    subset.append(i)
            remain = remain[1:]
        lf =  LF[subset]
        lr =  LR[subset]
        subset = np.append(subset,lf)
        subset = np.append(subset,lr)
        subset = np.unique(np.sort(subset))
        subset = np.array(subset)
        (target_lf,target_lr) = TransferToSKLsequence(np.array(LF),np.array(LR),subset)
        ST = convert_from_scikit_learn_to_dic_STRUT(estimator,thresh,C,Q,target_lf,target_lr,estimator.tree_.feature[subset])
        RF.append(ST)
    return RF

In [13]:
ind = np.arange(len(y))
np.random.shuffle(ind)
print ind

[121  27  13  48  53  57  87  73  81   1  30  65  40  74  54   6  51   0
  45  24 128 111  67  64  61  82  97  36  70 122 120  35  29 124  98  11
  26   8 106  95 123  47  17 146 145 139  94 144 114  55  59  66 103  31
  52  71  86 141 138 133  15  93  28 105 115 109  85 147  20  23  33  84
  79 126  99  63  38 140  43 135 107   7  77 113 148  49   5 132   4  75
  88  50  60  42  22  25  89  37  18 142 143 137  10  46  96 118 100  44
 131  62  92  41  76  72 117 119  69 127  16  58 101  83   3  19 134 102
 125 116 104 149  78  14 108  12   9  68  80 129  39  91  34 110  90 130
   2 112  32 136  21  56]


In [22]:
Xsource = X[ind[0:100],:]
ysource = y[ind[0:100]]
Xtarget = X[ind[100:],:]
ytarget = y[ind[100:]]

In [25]:
from time import time

In [46]:
t0 = time()
STRUCT(Xsource,ysource,Xtarget,ytarget,n_trees=50,C=3,verbos = False)
t1 = time()
print t1-t0

/Users/luca/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:251: DeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
/Users/luca/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:93: RuntimeWarning: divide by zero encountered in log10
/Users/luca/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:93: RuntimeWarning: invalid value encountered in multiply
/Users/luca/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:62: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/luca/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:64: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/luca/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:65: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/luca/anacon

0.82718706131


In [39]:
%lprun -s -f STRUCT -T SpeedStrut_Iris.txt STRUCT(Xsource,ysource,Xtarget,ytarget,n_trees=10,C=3,verbos = False) 
%cat SpeedStrut_Iris.txt

/Users/luca/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:251: DeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
/Users/luca/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:93: RuntimeWarning: divide by zero encountered in log10
/Users/luca/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:93: RuntimeWarning: invalid value encountered in multiply
/Users/luca/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:62: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/luca/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:64: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/luca/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:65: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/luca/anacon


*** Profile printout saved to text file u'SpeedStrut_Iris.txt'. 
Timer unit: 1e-06 s

Total time: 0.276265 s
File: <ipython-input-35-66a1765c970c>
Function: STRUCT at line 277

Line #      Hits         Time  Per Hit   % Time  Line Contents
   277                                           def STRUCT(Xsource,ysource,Xtarget,ytarget,n_trees,C,verbos = False):
   278                                               # Assumption: ysource has all the labels of the problem 
   279                                               #estimator = DecisionTreeClassifier(max_features='sqrt',random_state=0,max_depth=2)
   280         1           65     65.0      0.0      Estimator = RandomForestClassifier(max_features='sqrt',random_state=0,n_estimators=n_trees)
   281         1        69801  69801.0     25.3      Estimator = Estimator.fit(Xsource, ysource)
   282         1            3      3.0      0.0      RF = []
   283        11           21      1.9      0.0      for rf in range(Estimator.n_estimator

In [ ]:
[th, ql, qr, left, right] = threshold_selection(Xtarget,ytarget,np.array(P[i]),f,QL,QR,C,verbos) 
